<a href="https://colab.research.google.com/github/ricky-kiva/dl-deep-tf-cv-advanced/blob/main/1_l1_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Transfer Learning (Cats vs. Dogs)**



Import libraries

In [ ]:
import os

Download dataset

In [ ]:
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip" \
    -O "/tmp/cats-and-dogs.zip"

Unzip dataset

In [ ]:
import zipfile

local_zip = '/tmp/cats-and-dogs.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

Referencing source path

In [ ]:
DIR_SOURCE = '/tmp/PetImages'

DIR_SOURCE_CAT = os.path.join(DIR_SOURCE, 'Cat')
DIR_SOURCE_DOG = os.path.join(DIR_SOURCE, 'Dog')

Data exploration: check number of samples

Expected:
- Cat: 12501
- Dog: 12501

In [ ]:
print(f"There are {len(os.listdir(DIR_SOURCE_CAT))} images of cat.")
print(f"There are {len(os.listdir(DIR_SOURCE_DOG))} images of dogs.")

Create directories to store *training* & *test* data

In [ ]:
try:
    os.mkdir('/tmp/cats-v-dogs')
    os.mkdir('/tmp/cats-v-dogs/training')
    os.mkdir('/tmp/cats-v-dogs/testing')
    os.mkdir('/tmp/cats-v-dogs/training/cats')
    os.mkdir('/tmp/cats-v-dogs/training/dogs')
    os.mkdir('/tmp/cats-v-dogs/testing/cats')
    os.mkdir('/tmp/cats-v-dogs/testing/dogs')
except OSError:
    pass

Referencing *train* & *test* directories

In [ ]:
DIR_TRAIN = '/tmp/cats-v-dogs/training'
DIR_TEST = '/tmp/cats-v-dogs/testing'

DIR_TRAIN_CAT = os.path.join(DIR_TRAIN, 'cats')
DIR_TRAIN_DOG = os.path.join(DIR_TRAIN, 'dogs')

DIR_TEST_CAT = os.path.join(DIR_TEST, 'cats')
DIR_TEST_DOG = os.path.join(DIR_TEST, 'dogs')

Function: Split *training* & *test* set

In [ ]:
import random
from shutil import copyfile

def split_data(SOURCE, TRAIN, TEST, SPLIT_SIZE):
  files = []

  for fn in os.listdir(SOURCE):
    file_path = os.path.join(SOURCE, fn)
    if os.path.getsize(file_path) > 0:
      files.append(fn)
    else:
      print(fn + ' is zero length, so ignoring.')

  train_length = int(len(files) * SPLIT_SIZE)
  test_length = int(len(files) - train_length)

  shuffled_set = random.sample(files, len(files))
  train_set = shuffled_set[0:train_length]
  test_set = shuffled_set[train_length:]

  for fn in train_set:
    train_file_path = os.path.join(SOURCE, fn)
    destination = os.path.join(TRAIN, fn)
    copyfile(train_file_path, destination)

  for fn in test_set:
    test_file_path = os.path.join(SOURCE, fn)
    destination = os.path.join(TEST, fn)
    copyfile(test_file_path, destination)

Split *train* & *test* set

In [ ]:
SPLIT_SIZE = .9

split_data(DIR_SOURCE_CAT, DIR_TRAIN_CAT, DIR_TEST_CAT, SPLIT_SIZE)
split_data(DIR_SOURCE_DOG, DIR_TRAIN_DOG, DIR_TEST_DOG, SPLIT_SIZE)

Data exploration: Check *train* & *test* set length

Expected:
- Train Cat: 11250
- Train Dog: 11250
- Test Cat: 1250
- Test Dog: 1250

In [ ]:
print("Number of training cat images", len(os.listdir(DIR_TRAIN_CAT)))
print("Number of training dog images", len(os.listdir(DIR_TRAIN_DOG)))
print("Number of testing cat images", len(os.listdir(DIR_TEST_CAT)))
print("Number of testing dog images", len(os.listdir(DIR_TEST_DOG)))

Data augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(DIR_TRAIN,
                                                    batch_size=100,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(DIR_TEST,
                                                              batch_size=100,
                                                              class_mode='binary',
                                                              target_size=(150, 150))

Preparing `InceptionV3` pre-trained model

Note:
- freeze existing layer so they aren't trained on the downstream task (cats & dogs data)
- get reference to last layer `mixed7`, so some layers could be added after this last layer

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
import urllib.request

weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"

urllib.request.urlretrieve(weights_url, weights_file)

# instantiate the model
pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
                                include_top=False,
                                weights=None)

# load pre-trained weights
pre_trained_model.load_weights(weights_file)

# freeze the layers
for layer in pre_trained_model.layers:
  layer.trainable = False

Choose desired last layer (`mixed7`)

Note. Couple of consideration not to use up to the last layer:
- Layers become more specialized & detailed as it go deeper. If one's new task is somewhat related but not exactly the same, mixed7 would already capture lot of relevant information
- Save train time

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')

print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

Add layers to train on Cats & Dogs data

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras import layers

# flatten output of last layer
x = layers.Flatten()(last_output)
# add fully connected layers
x = layers.Dense(1024, activation='relu')(x)
# add final sigmoid layer to classify
x = layers.Dense(1, activation='sigmoid')(x)

# concatenate 2 layer
model = Model(pre_trained_model.input, x)

Check concatenated model

In [ ]:
model.summary()

Set train parameters

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['acc'])

Train the model

In [ ]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=5,
                    verbose=1)

Visualize *train* & *validation* accuracy

In [ ]:
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc))

plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

### Predict on a test image

You can upload any image and have the model predict whether it's a dog or a cat.
- Find an image of a dog or cat
- Run the following code cell.  It will ask you to upload an image.
- The model will print "is a dog" or "is a cat" depending on the model's prediction.

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():

    path = '/content/' + fn
    img = image.load_img(path, target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    image_tensor = np.vstack([x])
    classes = model.predict(image_tensor)
    print(classes)
    print(classes[0])
    if classes[0]>0.5:
        print(fn + " is a dog")
    else:
        print(fn + " is a cat")